In [ ]:

import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers importDense, Dropout, GlobalAveragePooling2D, SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
import keras_tuner as kt
from keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import CategoricalCrossentropy
import seaborn as sns

2024-12-17 05:54:56.195207: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 05:54:56.195338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 05:54:56.225781: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 05:54:56.294024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-17 05:54:57.332303: W tensorflow/compiler/tf2

In [4]:
major_class = pd.read_csv('major_class.csv')

In [5]:
minor_class = pd.read_csv("minor_class.csv")

In [6]:
disease_class = pd.concat([major_class, minor_class], axis=0, ignore_index=True)

In [7]:
disease_class_encoded = pd.get_dummies(disease_class, columns=['Label'], prefix='Label')

disease_class_encoded.head()

,Pixel_0,Pixel_1,Pixel_2,Pixel_3,Pixel_4,Pixel_5,Pixel_6,Pixel_7,Pixel_8,Pixel_9,...,Pixel_49150,Pixel_49151,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7
0,0,2,1,0,2,1,0,2,0,0,...,2,1,False,False,True,False,False,False,False,False
1,66,51,39,74,56,42,80,61,45,87,...,17,15,False,False,True,False,False,False,False,False
2,225,136,152,224,135,152,227,136,152,228,...,137,135,False,False,True,False,False,False,False,False
3,219,130,128,222,132,133,223,134,136,223,...,97,92,False,False,True,False,False,False,False,False
4,125,162,203,124,162,201,121,159,198,121,...,159,197,False,False,True,False,False,False,False,False


In [8]:
X = disease_class_encoded.drop(columns=['Label_0', 'Label_1', 'Label_2', 'Label_3', 'Label_4', 'Label_5', 'Label_6', 'Label_7']).values
X = X.reshape(-1, 128, 128, 3) / 255.0

y = disease_class_encoded[['Label_0', 'Label_1', 'Label_2', 'Label_3', 'Label_4', 'Label_5', 'Label_6', 'Label_7']].values

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (10560, 128, 128, 3)
Shape of X_val: (2640, 128, 128, 3)
Shape of y_train: (10560, 8)
Shape of y_val: (2640, 8)


In [ ]:
base_model = ResNet50(
    weights='imagenet',        
    include_top=False,         
    input_shape=(128, 128, 3) 
)


base_model.trainable = False

resnet_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),   
    Dense(128, activation='relu'), 
    Dropout(0.5),        
    Dense(8, activation='softmax')  
])

resnet_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

resnet_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 8)                 1032      
                                                                 
Total params: 23851016 (90.98 MB)
Trainable params: 263304 (1.00 MB)
Non-trainable params: 23587712 (89.98 MB)
___________

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',   
    patience=20,           
    restore_best_weights=True, 
    verbose=1
)

In [15]:
history = resnet_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,        
    batch_size=32,
    callbacks=[early_stopping],  
    verbose=1
)

Epoch 1/200
330/330 [==============================] - 6s 18ms/step - loss: 1.6591 - accuracy: 0.3082 - val_loss: 1.6448 - val_accuracy: 0.3716
Epoch 2/200
330/330 [==============================] - 4s 13ms/step - loss: 1.6557 - accuracy: 0.3154 - val_loss: 1.6432 - val_accuracy: 0.3519
Epoch 3/200
330/330 [==============================] - 4s 13ms/step - loss: 1.6516 - accuracy: 0.3216 - val_loss: 1.6400 - val_accuracy: 0.3701
Epoch 4/200
330/330 [==============================] - 4s 12ms/step - loss: 1.6506 - accuracy: 0.3160 - val_loss: 1.6372 - val_accuracy: 0.3652
Epoch 5/200
330/330 [==============================] - 4s 12ms/step - loss: 1.6474 - accuracy: 0.3240 - val_loss: 1.6349 - val_accuracy: 0.3477
Epoch 6/200
330/330 [==============================] - 4s 12ms/step - loss: 1.6458 - accuracy: 0.3250 - val_loss: 1.6325 - val_accuracy: 0.3803
Epoch 7/200
330/330 [==============================] - 4s 12ms/step - loss: 1.6455 - accuracy: 0.3336 - val_loss: 1.6333 - val_accuracy:

In [17]:
y_pred = resnet_model.predict(X)

y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y, axis=1)


accuracy = accuracy_score(y_true_classes, y_pred_classes)
accuracy

413/413 [==============================] - 4s 9ms/step


0.4593181818181818